## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key


# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
# city_data_df = pd.read_csv(r'C:/UCD/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv')
# city_data_df = pd.read_scv(r'C:/Users/s.etoumi/Desktop/UCD/World_Weather_Analysis/Weather_Database.csv')
city_data_df = pd.read_csv('/Users/s.etoumi/Desktop/UCD/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv')
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Garowe,8.4054,48.4845,74.86,few clouds,58,16,10.31,SO,2022-11-14 05:13:17
1,1,Rikitea,-23.1203,-134.9692,74.50,overcast clouds,87,98,12.82,PF,2022-11-14 05:13:18
2,2,Vila Velha,-20.3297,-40.2925,76.95,overcast clouds,91,100,11.99,BR,2022-11-14 05:13:19
3,3,New Norfolk,-42.7826,147.0587,56.91,moderate rain,75,100,4.00,AU,2022-11-14 05:13:19
4,4,Ushuaia,-54.8000,-68.3000,55.06,clear sky,50,0,9.22,AR,2022-11-14 05:13:20


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 95
What is the maximum temperature you would like for your trip? 75


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country,Date


In [ ]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head()


In [ ]:
preferred_cities_df.count()

In [ ]:
preferred_cities_df.head()

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()
hotel_df.count()

In [ ]:
hotel_df.head()

In [ ]:
# 8a. Create the output File (CSV)

output_data_file = "Weather_Database/hotels_df.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

# 11b. Display the figure
